In [30]:
import os

import torch
import urdfpy

import torchcontrol as toco
from torchcontrol.transform import Rotation as R
from torchcontrol.transform import Transformation as T
from polymetis.utils.data_dir import get_full_path_to_urdf

In [31]:
# experiment hyperparams
NUM_SAMPLES = 100
TIME_TO_GO = 4

# env hyperparams
URDF_PATH = get_full_path_to_urdf("franka_panda/panda_arm.urdf")
EE_NAME = "panda_link8"

HZ = 1000
NUM_DOFS = 7
UPPER_JOINT_LIMITS = [2.80, 1.66, 2.80, -0.17, 2.80, 3.65, 2.80]
LOWER_JOINT_LIMITS = [-2.80, -1.66, -2.80, -2.97, -2.80, 0.08, -2.80]
UPPER_EE_POS_LIMITS = [1.0, 0.4, 1.0]
LOWER_EE_POS_LIMITS = [0.1, -0.4, -0.05]
EE_ORI_RANGE = 1

REST_POSE = [-0.1394, -0.0205, -0.0520, -2.0691, 0.05059, 2.0029, -0.9168]
REST_ORI = [0.9383,  0.3442, -0.0072, -0.0318]

In [32]:
# sampling functions
joint_pos_hi = torch.Tensor(UPPER_JOINT_LIMITS)
joint_pos_lo = torch.Tensor(LOWER_JOINT_LIMITS)
ee_pos_hi = torch.Tensor(UPPER_EE_POS_LIMITS)
ee_pos_lo = torch.Tensor(LOWER_EE_POS_LIMITS)

joint_pos_rest = torch.Tensor(REST_POSE)
ee_quat_rest = torch.Tensor(REST_ORI)

def sample_joint_pos():
    joint_pos_diff = joint_pos_hi - joint_pos_lo
    return joint_pos_lo + torch.rand(NUM_DOFS) * joint_pos_diff

def sample_ee_pose():
    ee_pos_diff = ee_pos_hi - ee_pos_lo
    ee_pos_sampled = ee_pos_lo + torch.rand(3) * ee_pos_diff

    ee_rot_sampled = R.from_rotvec(EE_ORI_RANGE * torch.randn(3)) * R.from_quat(ee_quat_rest)
    
    return T.from_rot_xyz(ee_rot_sampled, ee_pos_sampled)

# robot model
robot_model = toco.models.RobotModelPinocchio(URDF_PATH, EE_NAME)

# collect data
trajectories = []
for _ in range(NUM_SAMPLES):
    # sample start & destination
    joint_pos_start = sample_joint_pos()
    ee_pose_target = sample_ee_pose()
    
    # plan trajectory
    waypoints = toco.planning.generate_cartesian_target_joint_min_jerk(
        joint_pos_start, ee_pose_target, TIME_TO_GO, HZ, robot_model
    )
    trajectories.append(waypoints)

In [33]:
# visualize
len(trajectories)

100